In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002293825149536133
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 20.727931986997167
200000 31.355445576586188
300000 31.596358473876062
400000 30.436281057301926
500000 32.175686205727956
600000 30.638314322824446
700000 29.982698797047316
800000 30.517087134741754
900000 25.26223229158505
1000000 17.24653875894252
1100000 30.25776877460497
1200000 29.998321373650437
1300000 30.06435384608025
1400000 31.913383235888382
1500000 30.541277610849278
1600000 30.317562522268034
1700000 30.80638907482526
1800000 28.441496565122083
1900000 13.766251467298023
2000000 30.032964195899417
2100000 29.613961643114077
2200000 29.541930110299337
2300000 30.823865473548622
2400000 29.602888054418784
2500000 29.295083466872075
2600000 28.985357265881895
2700000 28.877128379719615
2800000 12.688704653817268
2900000 28.912060045244854
300

29400000 21.471668829309767
29500000 29.85663742204528
29600000 30.465959577732146
29700000 28.02779696034482
29800000 28.42311623251377
29900000 25.61797520758015
30000000 11.732907305068991
30100000 28.541773358979356
30200000 25.302832458880893
30300000 27.3650440581032
30400000 29.808021092279837
30500000 29.088369173017536
30600000 31.202841962824298
30700000 15.133273084023715
30800000 28.580673817733825
30900000 31.841288959027274
31000000 30.359399856835626
31100000 31.639078714784034
31200000 15.246531371954651
31300000 30.652913967895692
31400000 31.597071967162908
31500000 29.83764540484007
31600000 30.392145568758036
31700000 26.61273069712651
31800000 15.260340885224196
31900000 25.125434102394358
32000000 26.71855812558409
32100000 30.546929888247504
32200000 28.659506220704984
32300000 31.911095143358438
32400000 31.464318861432695
32500000 15.10450016481282
32600000 32.13523370528483
32700000 30.621678688777802
32800000 33.26806321903254
32900000 30.3750743753552
330000

59100000 7.601935594184458
59200000 12.661163591577541
59300000 15.535287634388595
59400000 11.324588982271987
59500000 6.445616484452039
59600000 12.140474369415124
59700000 11.903513406707752
59800000 12.834111096935487
59900000 13.377349413028101
60000000 20.51045036222546
60100000 14.706490006442651
60200000 13.674072887632072
60300000 19.14742953132342
60400000 12.045524727847155
60500000 6.847944952363431
60600000 11.977850826570426
60700000 14.653733736906938
60800000 15.019241141983718
60900000 8.496379789538969
61000000 17.22138075816204
61100000 16.24275585876435
61200000 12.746615914647274
61300000 6.3071662187700275
61400000 19.290153617859872
61500000 13.609885971319503
61600000 13.912999008031862
61700000 13.066448405454697
61800000 13.878461107860529
61900000 12.962228161459104
62000000 15.549059406903206
62100000 15.577036027078892
62200000 20.00948981717028
62300000 5.712440115072708
62400000 11.359220341894371
62500000 13.648079713032223
62600000 11.497376375449887
62

88800000 24.391462195845627
88900000 25.440811719971375
89000000 25.671167945978105
89100000 27.754330148131093
89200000 26.109698755802643
89300000 23.79703453513201
89400000 23.700876193506947
89500000 24.17549297106754
89600000 23.809600943183998
89700000 19.936796951070438
89800000 15.474977452899347
89900000 14.933756091725646
90000000 18.05365076493049
90100000 13.79149554488112
90200000 23.119242874769558
90300000 23.524086030198486
90400000 24.62766625646126
90500000 24.261854162877807
90600000 24.06271857948465
90700000 26.121212204508605
90800000 27.6855920997836
90900000 25.85347027992949
91000000 24.657407503970187
91100000 23.918742993854394
91200000 22.64729360692248
91300000 22.61663953113361
91400000 23.164637411426764
91500000 24.186044132907707
91600000 11.675259625739136
91700000 17.764717402124848
91800000 14.652799575966398
91900000 13.287332386520587
92000000 22.33334460985755
92100000 22.989988320964382
92200000 22.775285926638563
92300000 23.058327229113388
9240

117900000 31.44915548867577
118000000 30.498969296465113
118100000 27.877234826477807
118200000 25.31603771145154
118300000 24.518725800273604
118400000 23.777551981832314
118500000 24.450582601576144
118600000 24.24119147728343
118700000 22.778373560623738
118800000 23.06997451026628
118900000 16.525091607920967
119000000 3.7523258839863933
119100000 15.733940293862586
119200000 23.15335000021862
119300000 24.77880880559888
119400000 23.923126353451096
119500000 25.61379435004431
119600000 23.572383339303347
119700000 24.756976488592624
119800000 24.67054929400726
119900000 24.746404458847078
120000000 21.681047951800096
120100000 21.553692575333493
120200000 19.748653986837137
120300000 19.97302121815809
120400000 21.112634255226467
120500000 19.775333453093825
120600000 18.779813929052946
120700000 14.027023968748354
120800000 3.759141167922775
120900000 11.178983663454016
121000000 19.47487570347653
121100000 20.610246144063165
121200000 19.66037765090937
121300000 20.7698144281976

...

319900000 4.103111122719616
320000000 5.653835289143293
320100000 5.7262892758805295
320200000 5.315075298381282
320300000 5.658730004574358
320400000 5.8584818797454865
320500000 2.2440846187309624
320600000 2.7089867369104033
320700000 5.66783527204411
320800000 5.879185984829723
320900000 6.315630206118077
321000000 6.187714571646244
321100000 6.289662992162701
321200000 6.6501834379795195
321300000 5.9114101519952715
321400000 6.446240227548072
321500000 6.0311945185272
321600000 2.9592035753911734
321700000 4.623511109852152
321800000 5.640232237865467
321900000 5.8339936703214175
322000000 5.7153993345249
322100000 5.580491118794649
322200000 5.193790850506067
322300000 3.530916076103972
322400000 1.7210393661593695
322500000 5.913955370725858
322600000 5.918708416618748
322700000 6.265912538993984
322800000 6.057228516252726
322900000 6.230569803150647
323000000 6.494596578427613
323100000 5.469190246003233
323200000 6.600390661448973
323300000 6.123977260191868
323400000 2.4342

348100000 4.190120194501409
348200000 3.349923897503495
348300000 3.3658526579143904
348400000 4.033754567480269
348500000 3.3557569412490627
348600000 4.806173758975946
348700000 4.480588153015064
348800000 4.67064879772681
348900000 3.5123934844049782
349000000 4.280507239984254
349100000 3.5412682920550105
349200000 3.0307466167049473
349300000 4.558137415366815
349400000 2.9597592392812966
349500000 4.4164439409246405
349600000 4.604510829135557
349700000 4.798444395975322
349800000 3.4360503531897066
349900000 3.9259248081390963
350000000 3.691027955419958
350100000 2.8090133591020505
350200000 3.9018004253183403
350300000 2.908652744098067
350400000 4.302568537567062
350500000 4.513878913491299
350600000 4.549221794657389
350700000 3.3406051627464093
350800000 4.092364767958912
350900000 3.9609286175620557
351000000 2.6590010807078808
351100000 3.9410301038189997
351200000 2.867231008555849
351300000 4.155099114337179
351400000 4.641630559653332
351500000 4.206937522039095
351600

376700000 1.4570815961652797
376800000 1.5937798120118811
376900000 1.3725725688315864
377000000 1.3132077057252824
377100000 0.8872020305290819
377200000 1.2441218702711627
377300000 0.7814234024614594
377400000 1.2269927006159722
377500000 0.9963648361456924
377600000 1.4766704231517713
377700000 1.4466429708611237
377800000 2.0492661548677815
377900000 1.2938263586145342
378000000 0.9271427300604482
378100000 1.2041905705482496
378200000 0.7211776351283292
378300000 1.2282188509509397
378400000 1.0983423635291358
378500000 2.210171883220881
378600000 1.3403387313751611
378700000 1.5195757194242012
378800000 2.0551323568605184
378900000 0.9172810244057609
379000000 1.5975111701655818
379100000 0.8416119371546208
379200000 1.5033791132711705
379300000 0.9818077120401991
379400000 1.4076672039619984
379500000 1.3590232926667365
379600000 1.4907984203982219
379700000 1.6770196280095875
379800000 1.1107562809733251
379900000 1.2266083086087538
380000000 0.7314706978733838
380100000 1.639

403900000 0.7506259529035257
404000000 0.45135142217329144
404100000 0.7004241307038432
404200000 0.7984404672338499
404300000 0.7757069791692496
404400000 0.4985366860801478
404500000 0.5414104197781285
404600000 0.8465731785834035
404700000 0.6531219639848007
404800000 0.4811696462981005
404900000 0.39695170280984043
405000000 0.6235400488022201
405100000 0.79685677794832
405200000 1.1052900704515676
405300000 0.3091438043389619
405400000 0.5473466582138174
405500000 0.6534955581986707
405600000 0.6365516926880379
405700000 0.5003322250427571
405800000 0.3364729395102561
405900000 0.6155665041645766
406000000 0.8219512534550429
406100000 0.6143749869805972
406200000 0.31266979018448127
406300000 0.5655836083860761
406400000 0.6448004259538352
406500000 0.7041425086645021
406600000 0.6251713930954371
406700000 0.3675318476047218
406800000 0.6538907012135385
406900000 0.4332994579666522
407000000 0.10761943851596117
407100000 0.11038695162749622
407200000 0.10977291842164266
407300000 

430100000 0.10022123686560988
430200000 0.06741851465171907
430300000 0.09064037899125615
430400000 0.0617555013120075
430500000 0.09143880674015151
430600000 0.06860271126326256
430700000 0.08391217235759894
430800000 0.08132567256623838
430900000 0.06627446555923422
431000000 0.08231902989077568
431100000 0.05711604435946545
431200000 0.08008504803440636
431300000 0.05493197885746757
431400000 0.07328599147345623
431500000 0.05887270544402798
431600000 0.06683796199848255
431700000 0.06757895454881588
431800000 0.055094083177622814
431900000 0.069944496229738
432000000 0.04999481834833158
432100000 0.07310086008322239
432200000 0.05127812313774228
432300000 0.06042594180706143
432400000 0.05203369112847249
432500000 0.05154949583832423
432600000 0.05637167884369691
432700000 0.044017601640833744
432800000 0.055899423135131604
432900000 0.04646836608617505
433000000 0.05155919724179308
433100000 0.04030077504613002
433200000 0.053293598228325446
433300000 0.04512710612996088
433400000